# OpenSky and COVID-19 data exploration

In this notebook we will do some basic exploration with OpenSky and COVID-19 data, and derive some understanding to aid us in developing a data model, and cleaning and aggregation.

In [1]:
import pandas as pd
import configparser
import os
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType

config = configparser.ConfigParser()
config.read('config.cfg')

%load_ext autoreload
%autoreload 2

In [2]:
# Read in the data
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "com.amazonaws:aws-java-sdk:1.7.3,org.apache.hadoop:hadoop-aws:2.7.3") \
        .getOrCreate()

# Opensky dataset (Local)
df_os_raw = spark.read.format('csv').option('header', 'true').load(config['DATA']['OPENSKY_FLIGHTS'])

# Airport codes dataset (Local)
df_airports_raw = spark.read.format('csv').option('header', 'true').load(config['DATA']['AIRPORT_CODES'])

# Country codes dataset (Local)
df_iban = spark.read.format('json').load(config['DATA']['COUNTRY_CODES'])

# John Hopkins University COVID-19 dataset (Local)
df_covid_raw = spark.read.format('json').load(config['DATA']['COVID19_CASES'])

### OpenSky Dataset

In [3]:
# OpenSky data sample
df_os_raw.limit(5).toPandas()

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,SXI1963,None,7c1ace,VH-FKW,F50,YPAD,WSSL,2019-12-31 00:07:07+00:00,2020-01-01 09:01:00+00:00,2020-01-01 00:00:00+00:00,-34.95975300417109,138.4747123718262,609.6,1.3885345458984382,103.84919764631891,182.88
1,THY183,None,4ba9c1,TC-JNA,A332,None,LTBW,2019-12-31 00:25:26+00:00,2020-01-01 07:48:41+00:00,2020-01-01 00:00:00+00:00,42.134523553363344,26.99328045512355,7924.8,41.147848872815146,28.7096474387429,716.2800000000003
2,CES771,MU771,781858,B-30CW,A359,YSSY,EHAM,2019-12-31 01:29:25+00:00,2020-01-01 04:08:28+00:00,2020-01-01 00:00:00+00:00,-33.924545288085945,151.17013736647,304.8,52.31435743428893,4.765359061104911,-160.02
3,AMX037,None,0d09e5,XA-ADC,B789,SAEZ,LEBL,2019-12-31 03:08:58+00:00,2020-01-01 10:05:29+00:00,2020-01-01 00:00:00+00:00,-34.82531014135327,-58.51850509643555,0.0,41.38957214355469,2.349950617009944,533.4
4,ACA43,None,c0173f,C-FIUW,B77W,CYYZ,CYYZ,2019-12-31 03:13:16+00:00,2020-01-01 10:25:51+00:00,2020-01-01 00:00:00+00:00,43.69465255737305,-79.63428763456125,0.0,43.67595291137695,-79.61098959279616,259.08


In [4]:
# explore some statistics from Opensky dataset
os_num_recs = df_os_raw.count()
os_null_origins = df_os_raw.where(col('origin').isNull()).count()
os_null_destinations = df_os_raw.where(col('destination').isNull()).count()
print(f"Number of records    : {os_num_recs:8d}")
print(f"Missing origins      : {os_null_origins:8d}")
print(f"Missing destinations : {os_null_destinations:8d}")

Number of records    : 14889687
Missing origins      :  3882199
Missing destinations :  2846499


### Airports Dataset

In [5]:
# Airports data sample
df_airports_raw.limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [6]:
# explore statistics from airport dataset
airports_num_recs = df_airports_raw.count()
airport_types = df_airports_raw.select('type').distinct().toPandas()['type']
airports_num_countries = df_airports_raw.select('iso_country').distinct().count()
print(f"Number of records    : {airports_num_recs:8d}")
print(f"Number of countries  : {airports_num_countries:8d}")
print(f"Airport types        : {', '.join(airport_types)}")

Number of records    :    55075
Number of countries  :      244
Airport types        : large_airport, balloonport, seaplane_base, heliport, closed, medium_airport, small_airport


## Country IBAN Dataset

In [7]:
df_iban.limit(5).toPandas()

,Code,Name
0,AF,Afghanistan
1,AX,Åland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa


### JHU COVID-19 Dataset

In [8]:
# JHU COVID-19 data sample
df_covid_raw.limit(5).toPandas()

,admin2,code3,confirmed,country_region,date,deaths,fips,iso2,iso3,latitude,longitude,province_state,recovered,uid
0,,16,0,US,2020-01-22,0,60,AS,ASM,-14.2710,-170.1320,American Samoa,NaN,16
1,,316,0,US,2020-01-22,0,66,GU,GUM,13.4443,144.7937,Guam,NaN,316
2,,580,0,US,2020-01-22,0,69,MP,MNP,15.0979,145.6739,Northern Mariana Islands,NaN,580
3,,630,0,US,2020-01-22,0,72,PR,PRI,18.2208,-66.5901,Puerto Rico,NaN,630
4,,850,0,US,2020-01-22,0,78,VI,VIR,18.3358,-64.8963,Virgin Islands,NaN,850


In [9]:
# explore statistics
covid_num_recs = df_covid_raw.count()
covid_num_regions = df_covid_raw.select('country_region').distinct().count()
covid_num_dates = df_covid_raw.select('date').distinct().count()
print(f"Number of records    : {covid_num_recs:8d}")
print(f"Number of countries  : {covid_num_regions:8d}")
print(f"Number of dates      : {covid_num_dates:8d}")

Number of records    :   450597
Number of countries  :      177
Number of dates      :      129
